<a href="https://colab.research.google.com/github/HappyJJins/Jeju_CreditCard/blob/master/DataExploring02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive

ROOT = '/content/drive'
drive.mount(ROOT)

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from os.path import join

MY_GOOGLE_DRIVE_PATH = '/content/drive/My Drive/Colab Notebooks/Jeju'
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
from google.cloud import bigquery

import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

# 평가 메트릭
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 시각화
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt  # Matlab-style plotting

# 통계
from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from itertools import product

In [6]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Jeju/data/jeju_data_ver1/201901-202003.csv')
train.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [7]:
data = train.copy()

In [ ]:
data['STD_CLSS_NM'].unique()

array(['건강보조식품 소매업', '골프장 운영업', '과실 및 채소 소매업', '관광 민예품 및 선물용품 소매업',
       '그외 기타 스포츠시설 운영업', '그외 기타 종합 소매업', '기타 대형 종합 소매업', '기타 외국식 음식점업',
       '기타 주점업', '기타음식료품위주종합소매업', '마사지업', '비알콜 음료점업', '빵 및 과자류 소매업',
       '서양식 음식점업', '수산물 소매업', '슈퍼마켓', '스포츠 및 레크레이션 용품 임대업', '여관업', '욕탕업',
       '육류 소매업', '일반유흥 주점업', '일식 음식점업', '전시 및 행사 대행업', '중식 음식점업',
       '차량용 가스 충전업', '차량용 주유소 운영업', '체인화 편의점', '피자 햄버거 샌드위치 및 유사 음식점업',
       '한식 음식점업', '호텔업', '화장품 및 방향제 소매업', '휴양콘도 운영업', '여행사업', '자동차 임대업',
       '면세점', '버스 운송업', '택시 운송업', '기타 수상오락 서비스업', '내항 여객 운송업',
       '그외 기타 분류안된 오락관련 서비스업', '정기 항공 운송업'], dtype=object)

In [8]:
data_1903, data_2003 = data[data['REG_YYMM']==201903], data[data['REG_YYMM']==202003]

In [13]:
diff_03 = pd.concat([data_1903.groupby('STD_CLSS_NM').AMT.sum(), data_2003.groupby('STD_CLSS_NM').AMT.sum()], axis=1, keys=['1903_AMT','2003_AMT'])
diff_03['diff'] = (diff_03['1903_AMT']-diff_03['2003_AMT'])/diff_03['1903_AMT']
diff_03 = diff_03.reset_index()
diff_03.head()

,STD_CLSS_NM,1903_AMT,2003_AMT,diff
0,건강보조식품 소매업,6307887107,5753676884,0.087860
1,골프장 운영업,42108966385,39398851940,0.064360
2,과실 및 채소 소매업,38415185909,42491078825,-0.106101
3,관광 민예품 및 선물용품 소매업,1018056196,463741220,0.544484
4,그외 기타 분류안된 오락관련 서비스업,2005500,1580200,0.212067


In [14]:
diff_03 = diff_03.sort_values(by='diff', ascending=False).reset_index(drop=True)
diff_03

,STD_CLSS_NM,1903_AMT,2003_AMT,diff
0,여행사업,8066920544,420268181,0.947902
1,정기 항공 운송업,44892792670,3314992032,0.926158
2,면세점,29559210069,3933663113,0.866923
3,그외 기타 스포츠시설 운영업,231791910,42628600,0.816091
4,전시 및 행사 대행업,6226218379,1616742158,0.740333
5,버스 운송업,17037134504,5188894134,0.695436
6,휴양콘도 운영업,3378598195,1093634463,0.676305
7,호텔업,13817431974,5073633041,0.632809
8,내항 여객 운송업,3011608155,1324194320,0.560303
9,관광 민예품 및 선물용품 소매업,1018056196,463741220,0.544484


In [12]:
data_2002 = data[data['REG_YYMM']==202002]
diff_0203 = pd.concat([data_2002.groupby('STD_CLSS_NM').AMT.sum(), data_2003.groupby('STD_CLSS_NM').AMT.sum()], axis=1, keys=['2002_AMT','2003_AMT'])
diff_0203['diff'] = (diff_0203['2002_AMT']-diff_0203['2003_AMT'])/diff_0203['2002_AMT']
diff_0203 = diff_0203.reset_index()
diff_0203 = diff_0203.sort_values(by='diff', ascending=False).reset_index(drop=True)
diff_0203

,STD_CLSS_NM,2002_AMT,2003_AMT,diff
0,면세점,12425479612,3933663113,0.683420
1,휴양콘도 운영업,3317926290,1093634463,0.670386
2,정기 항공 운송업,9903281616,3314992032,0.665263
3,그외 기타 스포츠시설 운영업,107046140,42628600,0.601774
4,여행사업,1026582986,420268181,0.590615
5,호텔업,11017116476,5073633041,0.539477
6,버스 운송업,10329117050,5188894134,0.497644
7,전시 및 행사 대행업,2373964614,1616742158,0.318970
8,욕탕업,9496420875,6850723890,0.278599
9,마사지업,1008095963,735895101,0.270015


In [17]:
pd.concat([diff_03[['STD_CLSS_NM','diff']], diff_0203[['STD_CLSS_NM','diff']]], axis=1)

,STD_CLSS_NM,diff,STD_CLSS_NM,diff
0,여행사업,0.947902,면세점,0.683420
1,정기 항공 운송업,0.926158,휴양콘도 운영업,0.670386
2,면세점,0.866923,정기 항공 운송업,0.665263
3,그외 기타 스포츠시설 운영업,0.816091,그외 기타 스포츠시설 운영업,0.601774
4,전시 및 행사 대행업,0.740333,여행사업,0.590615
5,버스 운송업,0.695436,호텔업,0.539477
6,휴양콘도 운영업,0.676305,버스 운송업,0.497644
7,호텔업,0.632809,전시 및 행사 대행업,0.318970
8,내항 여객 운송업,0.560303,욕탕업,0.278599
9,관광 민예품 및 선물용품 소매업,0.544484,마사지업,0.270015


작년 동월 대비 가장 많이 감소한 업종 TOP10
1. 여행산업
2. 정기 항공 운송업
3. 면세점
4. 그외 기타 스포츠시설 운영업
5. 전시 및 행사 대행업
6. 버스 운송업
7. 휴양콘도 운영업
8. 호텔업
9. 관광 민에품 및 선물용품 소매업
10. 욕탕업

TOP10에 들어가지만 직전달보다는 오른 업종
* 내항 여객 운송업

In [45]:
sido_03 = data[data['REG_YYMM'].isin([201903,202003])].groupby(['CARD_SIDO_NM','REG_YYMM']).AMT.sum().reset_index()
sido_03 = sido_03[sido_03['REG_YYMM']==201903].merge(sido_03[sido_03['REG_YYMM']==202003], on='CARD_SIDO_NM')
sido_03 = sido_03[['CARD_SIDO_NM','AMT_x','AMT_y']]
sido_03.columns = ['CARD_SIDO_NM','AMT_1903','AMT_2003']
sido_03['diff'] = (sido_03['AMT_1903']-sido_03['AMT_2003'])/sido_03['AMT_1903']
sido_03 = sido_03.sort_values(by='diff', ascending=False).reset_index(drop=True)
sido_03

,CARD_SIDO_NM,AMT_1903,AMT_2003,diff
0,대구,207024164939,151649595832,0.267479
1,부산,314215793190,234904898665,0.252409
2,인천,219739553889,166915125854,0.240396
3,서울,1012627085366,789796734433,0.220052
4,제주,89420874275,71061095855,0.205319
5,경북,188082554647,150145066894,0.201707
6,울산,86076151130,70660254987,0.179096
7,대전,77582739516,65727041723,0.152814
8,충남,142463971579,124325365579,0.127321
9,강원,100109898251,88034516800,0.120621


In [50]:
sido_0203 = data[data['REG_YYMM'].isin([202002,202003])].groupby(['CARD_SIDO_NM','REG_YYMM']).AMT.sum().reset_index()
sido_0203 = sido_0203[sido_0203['REG_YYMM']==202002].merge(sido_0203[sido_0203['REG_YYMM']==202003], on='CARD_SIDO_NM')
sido_0203 = sido_0203[['CARD_SIDO_NM','AMT_x','AMT_y']]
sido_0203.columns = ['CARD_SIDO_NM','AMT_2002','AMT_2003']
sido_0203['diff'] = (sido_0203['AMT_2002']-sido_0203['AMT_2003'])/sido_0203['AMT_2002']
sido_0203 = sido_0203.sort_values(by='diff', ascending=False).reset_index(drop=True)
sido_0203

,CARD_SIDO_NM,AMT_2002,AMT_2003,diff
0,대구,170999808656,151649595832,0.113159
1,부산,250458378759,234904898665,0.062100
2,울산,74057851829,70660254987,0.045878
3,서울,810412676550,789796734433,0.025439
4,경북,153417402900,150145066894,0.021330
5,인천,167075935179,166915125854,0.000962
6,세종,12391784265,12472075388,-0.006479
7,대전,65010896084,65727041723,-0.011016
8,경남,204410924508,208445168889,-0.019736
9,제주,69580895083,71061095855,-0.021273


In [51]:
pd.concat([sido_03[['CARD_SIDO_NM','diff']], sido_0203[['CARD_SIDO_NM','diff']]], axis=1)

,CARD_SIDO_NM,diff,CARD_SIDO_NM,diff
0,대구,0.267479,대구,0.113159
1,부산,0.252409,부산,0.062100
2,인천,0.240396,울산,0.045878
3,서울,0.220052,서울,0.025439
4,제주,0.205319,경북,0.021330
5,경북,0.201707,인천,0.000962
6,울산,0.179096,세종,-0.006479
7,대전,0.152814,대전,-0.011016
8,충남,0.127321,경남,-0.019736
9,강원,0.120621,제주,-0.021273


작년 동월 대비 가장 많이 감소한 시도 TOP5
1. 대구
2. 부산
3. 인천
4. 서울
5. 경북

TOP10에 들어가지만 직전달보다는 오른 시도
* 제주

특정 시도의 업종별 소비금액 순위

In [59]:
def sido_clss(sido):
    df = data[data['CARD_SIDO_NM']==sido].groupby('STD_CLSS_NM').AMT.sum().reset_index()
    df = df.sort_values(by='AMT', ascending=False).reset_index(drop=True)
    return df

특정 업종의 시도별 소비금액 순위

In [66]:
def clss_sido(clss):
    df = data[data['STD_CLSS_NM']==clss].groupby('CARD_SIDO_NM').AMT.sum().reset_index()
    df = df.sort_values(by='AMT', ascending=False).reset_index(drop=True)
    return df

특정 시도의 2019년 3월 대비 2020년 3월 소비 감소율 순위

In [78]:
def sido_diff(sido):
    df = data[data['CARD_SIDO_NM']==sido]
    df_19 = df[df['REG_YYMM']==201903]
    df_20 = df[df['REG_YYMM']==202003]
    diff = pd.concat([df_19.groupby('STD_CLSS_NM').AMT.sum(), df_20.groupby('STD_CLSS_NM').AMT.sum()], axis=1, keys=['1903_AMT','2003_AMT'])
    diff['diff'] = (diff['1903_AMT']-diff['2003_AMT'])/diff['1903_AMT']
    diff = diff.reset_index()
    diff = diff.sort_values(by='diff', ascending=False).reset_index(drop=True)
    return diff

In [79]:
sido_diff('대구')

,index,1903_AMT,2003_AMT,diff
0,자동차 임대업,4747480,1.850000e+05,0.961032
1,호텔업,402020726,1.591280e+07,0.960418
2,기타 수상오락 서비스업,1603700,7.200000e+04,0.955104
3,그외 기타 스포츠시설 운영업,10987500,6.900000e+05,0.937201
4,전시 및 행사 대행업,236058620,2.649105e+07,0.887778
5,마사지업,15001600,2.194000e+06,0.853749
6,욕탕업,708719816,1.038175e+08,0.853514
7,관광 민예품 및 선물용품 소매업,38131020,7.100600e+06,0.813784
8,일반유흥 주점업,1295410205,2.829416e+08,0.781581
9,기타 주점업,384362160,1.144566e+08,0.702217


In [80]:
sido_diff('제주')

,index,1903_AMT,2003_AMT,diff
0,정기 항공 운송업,3.333231e+09,1.997219e+08,0.940082
1,휴양콘도 운영업,5.613140e+08,3.871976e+07,0.931019
2,여행사업,2.552282e+07,3.933600e+06,0.845879
3,관광 민예품 및 선물용품 소매업,3.173647e+08,5.173391e+07,0.836989
4,전시 및 행사 대행업,3.236805e+08,5.530123e+07,0.829149
5,면세점,7.585381e+09,2.590700e+09,0.658461
6,내항 여객 운송업,1.736811e+08,6.643240e+07,0.617504
7,마사지업,6.330000e+06,2.660000e+06,0.579779
8,호텔업,1.128726e+09,4.949414e+08,0.561504
9,여관업,5.772515e+08,2.672720e+08,0.536992


In [70]:
clss_sido('여행사업')

,CARD_SIDO_NM,AMT
0,서울,105087859446
1,강원,1776928180
2,경기,663212820
3,제주,535102735
4,부산,461080937
5,충남,195329500
6,전남,82440200
7,대구,55275600
8,인천,38842200
9,광주,34138944
